## Pandas and SQL - Continued

In [1]:
#Import pandas
import pandas as pd

### Reading in the NRI point  data
Below, we read the csv files into a Pandas dataFrame as we have in the past - with a few exceptions.

Pandas, like MS Access, will infer the data type from the values it's importing. However, we have some numeric fields that need to be imported as strings: the `recordid`, `fips`, `hydro`, `mhydro`, and `mlra` fields. To do this, we create a dictionary of field names and the field types we want to override. Any fields left of this list will get the default data types.

We will also set the recordid as the index for the dataFrame.

In [3]:
#Create the dataType dictionary
dtypeDict = {'recordid':'str',
             'fips':'str',
             'hydro':'str',
             'mhydro':'str',
             'mlra':'str'
            }

#Read in the data
dfPoint = pd.read_csv('../Data/nc_point.csv',
                      index_col='recordid',
                      dtype=dtypeDict)

In [4]:
#Show the data types
dfPoint.dtypes

nriptr        int64
stratum       int64
cluster       int64
xfact         int64
fips         object
hydro        object
mhydro       object
mlra         object
bailey       object
split       float64
impute        int64
impute87      int64
wifact        int64
wcfact        int64
urfact        int64
ukfact      float64
usleflag      int64
s5id         object
s5name       object
surftxt      object
txtmod       object
loslope       int64
hislope       int64
flood        object
ophase       object
tfact         int64
dtype: object

In [ ]:
#Have a quick look 
dfPoint.head()

Ok. Now it's your turn. Import the nc_trend.csv file. Set the following columns to be strings: `recordid`,`yr`,`landuse`,`broad`. (Others columns with nominal data should be strings, but this will suffice...). Also, as above, set the `recordid` column to be the index.

In [67]:
dtypeDict = {'recordid':'str',
             'yr':'str',
             'landuse':'str',
             'broad':'str'
            }

dfTrend = pd.read_csv("../Data/nc_trend.csv", dtype=dtypeDict, index_col='recordid')
dfTrend.dtypes

yr          object
landuse     object
broad       object
prime        int64
crp          int64
crpcov       int64
crpnum       int64
dblcrop      int64
irtyp        int64
irsor        int64
irsys        int64
ucfact     float64
upfact     float64
lenslop      int64
slope      float64
landcl      object
knoll        int64
wrotat       int64
usle       float64
ei         float64
eiwater    float64
eiwind     float64
aaweq      float64
dtype: object

OK, now we are read to analyse the data (and learn how Pandas does it...)

* First another example of an aggregate function: Lets count the number of samples and total area of each location within each county using the `dfPoint` dataFrame.

In [68]:
#Create the grouping object
grpCounty = dfPoint.groupby('fips')
type(grpCounty)

pandas.core.groupby.DataFrameGroupBy

In [6]:
#With this DataFrameGroupBy object we can apply different aggregate functions.
dfX = grpCounty['fips'].agg('count')
dfX.head()

fips
37001    369
37003    191
37005    314
37007    336
37009    445
Name: fips, dtype: int64

In [ ]:
#Sum up the xfact values and muliply by 10
dfX = grpCounty['xfact'].agg('sum')
dfX.head()

In [ ]:
#Or we can combine the aggregating functions into a single 
# command using a dictionary to define how we want to aggregate

#Create a dictionary of field names: aggregating functions
grpFunctions = {'fips':['count'],'xfact':['sum']}

#Apply them all at once
dfX = grpCounty['xfact'].agg(grpFunctions)
dfX.head()

## Transforming data
Pandas can pivot data too. Let's pivot our `dfTrend` table so that it moves the year values into columns and presents the value in the `broad` column (for each year). This is done with the Pandas `pivot` function. The `columns` parameter is where we specify the column on which we want to pivot our data, and the `values` parameter is where we specify the column from which we take the values,  

In [71]:
dfX = dfTrend.pivot(columns='yr',values='broad')
dfX.head()

yr,1982,1987,1992,1997
recordid,,,,
37001010001,5,5,8,8
37001010002,5,5,5,5
37001010003,5,5,5,5
37001010004,5,5,5,5
37001010005,5,5,5,8


## Joining tables


In [54]:
#Build the broad codes dataFrame
dataDict = {'codes':['1','2','3','4','5','6','7','8','9','10','11','12'],
            'description':['Cropland_cultivated',
                      'Cropland_noncultivated',
                      'Pastureland',
                      'Rangeland',
                      'Forest land',
                      'Other rural land',
                      'Urban and built-up land',
                      'Rural transportation',
                      'Small water areas',
                      'Census water',
                      'Federal land',
                      'Conservation reserve program (CRP) land']}
dfBroadCodes = pd.DataFrame(dataDict,dtype='str')
dfBroadCodes

,codes,description
0,1,Cropland_cultivated
1,2,Cropland_noncultivated
2,3,Pastureland
3,4,Rangeland
4,5,Forest land
5,6,Other rural land
6,7,Urban and built-up land
7,8,Rural transportation
8,9,Small water areas
9,10,Census water


In [72]:
#Join to the dfX dataFrame
dfY = pd.merge(left=dfTrend,
               right=dfBroadCodes,
               how='outer',
               left_on='broad',
               right_on='codes')

In [73]:
dfY

,yr,landuse,broad,prime,crp,crpcov,crpnum,dblcrop,irtyp,irsor,...,landcl,knoll,wrotat,usle,ei,eiwater,eiwind,aaweq,codes,description
0,1982,342,5,1.0,9.0,9.0,99.0,0.0,0.0,0.0,...,2E,0.0,0.0,0.00,0.0,0.0,0.0,0.0,5,Forest land
1,1987,342,5,1.0,9.0,9.0,99.0,0.0,0.0,0.0,...,2E,0.0,0.0,0.00,0.0,0.0,0.0,0.0,5,Forest land
2,1982,342,5,0.0,9.0,9.0,99.0,0.0,0.0,0.0,...,4E,0.0,0.0,0.00,0.0,0.0,0.0,0.0,5,Forest land
3,1987,342,5,0.0,9.0,9.0,99.0,0.0,0.0,0.0,...,4E,0.0,0.0,0.00,0.0,0.0,0.0,0.0,5,Forest land
4,1992,342,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4E,0.0,0.0,0.00,0.0,0.0,0.0,0.0,5,Forest land
5,1997,342,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4E,0.0,0.0,0.00,0.0,0.0,0.0,0.0,5,Forest land
6,1982,342,5,0.0,9.0,9.0,99.0,0.0,0.0,0.0,...,6E,0.0,0.0,0.00,0.0,0.0,0.0,0.0,5,Forest land
7,1987,342,5,0.0,9.0,9.0,99.0,0.0,0.0,0.0,...,6E,0.0,0.0,0.00,0.0,0.0,0.0,0.0,5,Forest land
8,1992,342,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6E,0.0,0.0,0.00,0.0,0.0,0.0,0.0,5,Forest land
9,1997,342,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6E,0.0,0.0,0.00,0.0,0.0,0.0,0.0,5,Forest land


In [65]:
#Re-pivot
dfTrend.pivot(columns='yr',values='broad')

yr,1982,1987,1992,1997
recordid,,,,
37001010001,5,5,8,8
37001010002,5,5,5,5
37001010003,5,5,5,5
37001010004,5,5,5,5
37001010005,5,5,5,8
37001010006,1,1,1,1
37001010007,5,5,5,1
37001010008,6,6,6,6
37001010009,8,8,8,8
